# System Dynamics and Equations of Motion

The first step we will take will be to derive the equations of motion of the system. As usual, the first step in the Lagrangian mechanics approach is to calculate the total kinetic energy of the system and the total potential energy of the system with respect to the generalized coordinates and their derivatives. We can do this using the Python package Sympy.

In [21]:
# Import everything we need from SymPy
import sympy as sym
import numpy as np
from sympy import Symbol, Matrix, Function, Derivative, N
from sympy import diff, simplify, sin, cos, solve, init_printing, symbols
init_printing() # This function will make the outputs of SymPy look prettier and be easier to read

Now we are going to define the parameters of the system as constants:

In [22]:
# Constants of the system
mp = 0.5      # Mass of the sphere on the top of the frame
ixx = 0.03    # Lateral mass moment of inertia of the gimbal
izz = 0.04    # Longitudinal mass moment of inertia of the gimbal
l = 1.5       # Distance from the center of the gimbal to the center of the sphere on top of the frame
g = 9.81      # Acceleration due to gravity
rotor_velocity = 100.0 # Fixed angular rate of the gimbal in rad/sec

Now we make symbols and functions. These are elements of SymPy and can be thought of as exactly the same as symbols (variables) and functions from math.

In [23]:
# Time is a symbol (variable)
t = Symbol('t')

# The generalized coordinates and the input torque are both functions of time.
# This means that they are initialized as Functions.
theta = Function('theta')
phi = Function('phi')
tau = Function('tau')

Now we will start to calculate the energies of the system. We can start by getting the kinetic energy of the mass on top of the frame. We can do this by:
1. Defining its position in terms of the generalized coordinates
2. Take the derivate of the position with respect to time to get the velocity
3. Calculate the kinetic energy from the velocity
4. Calculate the potential energy by observation

In [24]:
# Get the position of the mass
pos = Matrix([-l*sin(theta(t)),
              0.0,
              l*cos(theta(t))])

In [25]:
# Take the derivative of position with respect to time
vel = diff(pos,t)

In [26]:
# Get the kinetic energy of the mass from the velocity
mass_KE = 0.5 * mp * (vel.T @ vel)[0,0]

In [27]:
# Get the potential energy of the mass based on its position
mass_PE = mp*g*l*cos(theta(t))

No other parts of the frame or gimbal cage have inertia or mass. This means the only other component that has energy is the gimbal itself. Because it's center of mass doesn't change altitude and is at the origin, the only energy it has is rotational energy. We will calculate that now.

In [28]:
# The is the rotational rate of the gimbal
gimbal_rate = Matrix([Derivative(phi(t), t),
                      Derivative(theta(t), t)*cos(phi(t)),
                      rotor_velocity - Derivative(theta(t), t)*sin(phi(t))])

In [29]:
# This is the mass moment of inertia of the gimbal
I = Matrix([[ixx, 0.0, 0.0],
            [0.0, ixx, 0.0],
            [0.0, 0.0, izz]])

In [30]:
# Calculate the rotational energy of the gimbal
gimbal_RE = (0.5 * (gimbal_rate.T @ I @ gimbal_rate))[0, 0]

Now we calculate the lagrangian of the system via the formula:$$L=T-V$$ where $T$ is the total kinetic energy and $V$ is the total potential energy.

In [31]:
# Get the lagrangian
L = (mass_KE + gimbal_RE) - mass_PE
print("System Lagrangian:")
L = simplify(L)
N(L, 3) # This rounds floating point number to 3 places and then prints to the screen

System Lagrangian:


Finally, we get the equations of motion of the system via the formulas:
$$\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{\theta}} \right) - \frac{\partial L}{\partial \theta}=0$$
$$\frac{d}{dt} \left( \frac{\partial L}{\partial \dot{\phi}} \right) - \frac{\partial L}{\partial \phi}=\tau$$

In [32]:
# Get the first equation of motion
eq1 = diff(diff(L, Derivative(theta(t), t)), t) - diff(L, theta(t))

In [33]:
# Get the second equation of motion
eq2 = diff(diff(L, Derivative(phi(t), t)), t) - diff(L, phi(t)) - tau(t)

This form of equations of motion is not very helpful to us right now, so next we will place them in standard form. A standard form system of ordinary differential equations satisfy the following requirements
* All equations are exactly first order ordinary differential equations
* Each equation only has exactly one time derivative variable in it
* Each equation has the form $\frac{d x}{dt} = f(x,y,z,...)$

Because the equations of motion above are second order, we need to apply a trick to place them in standard form. Namely, we introduce new variables that increase the number of equations while reducing the order of each equation. For example, let's introduce two new variables, $\omega_{\theta}$ and $\omega_{\phi}$ that satisfy the standard form ordinary differential equations:
$$\frac{d \theta}{dt} = \omega_{\theta}$$
$$\frac{d \phi}{dt} = \omega_{\phi}$$
Then, by taking the derivative of both sides, we see that
$$\frac{d^2 \theta}{d t^2} = \frac{d \omega_{\theta}}{dt}$$
$$\frac{d^2 \phi}{d t^2} = \frac{d \omega_{\phi}}{dt}$$
Let's make this change of variables to our equations of motion right now.

In [35]:
# Make the new functions for change of variables
omega_theta = Function('omega_theta')
omega_phi = Function('omega_phi')

In [36]:
# Make the change of variables for equation 1
eq1 = eq1.subs({Derivative(theta(t), (t, 2)) : Derivative(omega_theta(t), t), 
                Derivative(phi(t), (t, 2))   : Derivative(omega_phi(t), t),
                Derivative(theta(t), t)      : omega_theta(t),
                Derivative(phi(t), t)        : omega_phi(t)})

In [37]:
# Repeat for equation 2
eq2 = eq2.subs({Derivative(theta(t), (t, 2)) : Derivative(omega_theta(t), t), 
                Derivative(phi(t), (t, 2))   : Derivative(omega_phi(t), t),
                Derivative(theta(t), t)      : omega_theta(t),
                Derivative(phi(t), t)        : omega_phi(t)})

The final step to place the equations in standard form is to ensure they have the form $\frac{d x}{dt} = f(x,y,z,...)$. We do this now.

In [17]:
# Solve the equations for Derivative(omega_theta(t), t) and Derivative(omega_phi(t), t)
soln = solve([eq1, eq2],
              Derivative(omega_theta(t), t),
              Derivative(omega_phi(t), t))

We now have a system of ordinary differential equations in standard from. Specifically, we have:
$$\frac{d \omega_{\theta}}{dt} = f_{\omega_{\theta}}\left( \omega_{\theta}, \omega_{\phi}, \theta, \phi, \tau \right)$$
$$\frac{d \omega_{\phi}}{dt} = f_{\omega_{\phi}}\left( \omega_{\theta}, \omega_{\phi}, \theta, \phi, \tau \right)$$
$$\frac{d \theta}{dt} = f_{\theta}\left( \omega_{\theta}, \omega_{\phi}, \theta, \phi, \tau \right)$$
$$\frac{d \phi}{dt} = f_{\phi}\left( \omega_{\theta}, \omega_{\phi}, \theta, \phi, \tau \right)$$

Let's now combine the right hand side of this system into a single vector.

In [18]:
# Build the equations of motion vector
f = Matrix([soln[Derivative(omega_theta(t), t)],
            soln[Derivative(omega_phi(t), t)],
            omega_theta(t),
            omega_phi(t)])

# Replace the functions of time with symbols
(v1, v2, v3, v4, v5) = symbols('v1, v2, v3, v4, v5')
f = f.subs({omega_theta(t) : v1,
            omega_phi(t) : v2,
            theta(t) : v3,
            phi(t) : v4,
            tau(t) : v5})
(omega_theta, omega_phi, theta, phi, tau) = symbols('omega_theta, omega_phi, theta, phi, tau')
f = f.subs({v1 : omega_theta,
            v2 : omega_phi,
            v3 : theta,\
            v4 : phi,
            v5 : tau})

# Simplify
f = simplify(f)

$f$ is our system model such that
$$
\begin{bmatrix}
\dot{\omega}_{\theta} \\
\dot{\omega}_{\phi} \\
\dot{\theta} \\
\dot{\phi} \\
\end{bmatrix} = f(\omega_{\theta}, \omega_{\phi}, \theta, \phi, \tau)
$$

In [19]:
N(f, 3)  # This rounds floating point number to 3 places and then prints to the screen

⎡-4.0⋅ωᵩ⋅ωₜₕₑₜₐ⋅sin(2⋅φ) + 1.6e+3⋅ωᵩ⋅cos(φ) + 2.94e+3⋅sin(θ)⎤
⎢───────────────────────────────────────────────────────────⎥
⎢                           2                               ⎥
⎢                    4.0⋅sin (φ) + 462.0                    ⎥
⎢                                                           ⎥
⎢               2                                           ⎥
⎢   0.167⋅ωₜₕₑₜₐ ⋅sin(2⋅φ) - 133.0⋅ωₜₕₑₜₐ⋅cos(φ) + 33.3⋅τ   ⎥
⎢                                                           ⎥
⎢                          ωₜₕₑₜₐ                           ⎥
⎢                                                           ⎥
⎣                            ωᵩ                             ⎦